In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import torch.nn as nn
import transformers
import tokenizers
from transformers import (BertModel, 
                          BertConfig, 
                          RobertaConfig, 
                          RobertaModel)
from tokenizers import ByteLevelBPETokenizer
import os

%matplotlib inline

In [2]:
bert_config = BertConfig.from_pretrained("bert-base-uncased")
bert = BertModel.from_pretrained("bert-base-uncased")

roberta_config = RobertaConfig.from_pretrained("roberta-base")
roberta_config.output_attentions = False
roberta_config.output_hidden_states = True
roberta_config.output_past = True

roberta = RobertaModel.from_pretrained("roberta-base", config=roberta_config)

In [3]:
def read_data(datadir):
    train = pd.read_csv(os.path.join(datadir, "train.csv"))
    test = pd.read_csv(os.path.join(datadir, "test.csv"))
    sample_submission = pd.read_csv(os.path.join(datadir, "sample_submission.csv"))

    return (train, test, sample_submission)

In [4]:
train, test, _ = read_data("../data/")

In [5]:
pos = train.iloc[27478]
text, st = pos.text, pos.selected_text

print(text)
print(st)

 Yay good for both of you. Enjoy the break - you probably need it after such hectic weekend  Take care hun xxxx
Yay good for both of you.


In [6]:
bpe_tokenizer = ByteLevelBPETokenizer(
            vocab_file="../pretrained/roberta-base-vocab.json",
            merges_file="../pretrained/roberta-base-merges.txt",
            lowercase=True,
            add_prefix_space=True,
        )

roberta_tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")

bert_tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

In [7]:
MAX_LEN = 96

ids = np.ones((1, MAX_LEN), dtype=np.int32)
attention_mask = np.zeros((1, MAX_LEN), dtype=np.int32)
token_type_ids = np.zeros((1, MAX_LEN), dtype=np.int32)

In [8]:
encoded = bpe_tokenizer.encode(text)

# encode_length = 1 + len(encoded.ids) + 2 + 1 + 1

# ids[0,:encode_length] = [0] + encoded.ids + [2, 2] + [1331] + [1]
# attention_mask[0,:encode_length] = [1] * encode_length

In [9]:
e = roberta_tokenizer.build_inputs_with_special_tokens(bpe_tokenizer.encode(text).ids, 
                                                      bpe_tokenizer.encode('positive').ids)

In [10]:
e = roberta_tokenizer.create_token_type_ids_from_sequences(bpe_tokenizer.encode(text).ids, 
                                                      bpe_tokenizer.encode('positive').ids)

In [58]:
MAX_LEN = 96

encoded = bpe_tokenizer.encode(text)

ids = torch.tensor([encoded.ids])
mask = torch.tensor([encoded.attention_mask])
token_type_ids = torch.tensor([encoded.type_ids])

In [59]:
outputs = roberta(ids, attention_mask=mask, token_type_ids=token_type_ids)

In [60]:
lengths = []
for _, row in train.iterrows():
    if isinstance(row.text, str):
        encoded = roberta_tokenizer.encode(row.text)
        lengths += [len(encoded.ids)]

AttributeError: 'list' object has no attribute 'ids'

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12,4))
sns.countplot(lengths, ax=ax)

print(f"Maximum token size: {max(lengths)}")

In [72]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


In [14]:

class Config:
    datadir = "../data"
    modelsdir = "../models"
    roberta_vocab = "../pretrained/roberta-base-vocab.json"
    roberta_merges = "../pretrained/roberta-base-merges.txt"


def initialize_tokenizer():
    print("Tokenizer getting loaded...")

    tokenizer = ByteLevelBPETokenizer(
        vocab_file=Config.roberta_vocab,
        merges_file=Config.roberta_merges,
        lowercase=True,
        add_prefix_space=True)

    vocab_size = tokenizer.get_vocab_size()
    print(f"Vocab size = {vocab_size:,}")

    return tokenizer

In [15]:
tokenizer = initialize_tokenizer()

Tokenizer getting loaded...
Vocab size = 50,265


In [16]:
class TweetModel(transformers.BertModel):
    def __init__(self, config):
        super(TweetModel, self).__init__(config)
        self.roberta = transformers.RobertaModel.from_pretrained("roberta-base", config=config)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(config.hidden_size*2, 2)

    def forward(self, ids, attention_mask, token_type_ids):
        _, _, out = self.roberta(ids, 
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)
        
        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.dropout(out)
        logits = self.linear(out)

        return logits

In [17]:
roberta_config.output_hidden_states = True
tweetmodel = TweetModel(roberta_config)

In [18]:
MAX_LEN = 96

encoded = bpe_tokenizer.encode(text)

ids = torch.tensor([encoded.ids])
attention_mask = torch.tensor([encoded.attention_mask])
token_type_ids = torch.tensor([encoded.type_ids])

logits = tweetmodel(ids, attention_mask, token_type_ids)

In [19]:
start, end = logits[:,:,0], logits[:,:,1]
ground = torch.zeros_like(start)
ground[:,5] = 1

In [20]:
loss = nn.CrossEntropyLoss()

# Process tweet

In [461]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


In [455]:
def process_tweet(tweet, selected_text, sentiment, tokenizer, max_len):
    sentiment_ids = {
            'positive': 1313, 
            'negative': 2430, 
            'neutral': 7974
    }
    
    # initializing ids, attention_mask, token_typel_ids
    ids = np.zeros((max_len), dtype=np.int32)
    attention_mask = np.zeros((max_len), dtype=np.int32)
    token_type_ids = np.zeros((max_len), dtype=np.int32)
    
    # removing extra spaces and encoding tweet
    tweet = " " + " ".join(tweet.split())
    selected_text = " ".join(selected_text.split())
    encoded_tweet = tokenizer.encode(tweet)
        
    # filling the ids and attention_mask
    ids_valid = [0] + [sentiment_ids[sentiment]] + [2, 2] + encoded_tweet.ids + [2]
    len_valid = len(ids_valid)
    attention_mask_valid = [1] * len_valid

    ids[:len_valid] = ids_valid
    attention_mask[:len_valid] = attention_mask_valid
    
    
    selected_text_len = len(selected_text)

    for idx, char in enumerate(tweet):
        if char == selected_text[0]:
            if tweet[idx:selected_text_len+idx] == selected_text:
                char_start = idx
                char_end = char_start + selected_text_len

    assert char_start is not None
    assert char_end is not None
    assert tweet[char_start:char_end] == selected_text
        
    for token_index, (offset_start, offset_end) in enumerate(encoded_tweet.offsets):
        if (char_start >= offset_start) and (char_start <= offset_end):
#             print(tweet[offset_start:offset_end], char_start, offset_start, offset_end, token_index)
            token_start = token_index
        if (char_end-1 >= offset_start) and (char_end <= offset_end):
#             print(tweet[offset_start:offset_end], char_end, offset_start, offset_end, token_index)
            token_end = token_index
        
    assert token_start is not None
    assert token_end is not None
#     print(bpe_tokenizer.decode(encoded_tweet.ids[token_start:(token_end+1)]))
#     assert selected_text.lower() in bpe_tokenizer.decode(encoded_tweet.ids[token_start:(token_end+1)])
    
    token_start += 4
    token_end += 4
    
    return {
        "ids": ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
        "token_start": token_start,
        "token_end": token_end,
        "tweet": tweet,
        "selected_text": selected_text
    }
    

    

In [468]:
idx = 1495
record = train.iloc[idx]
tweet, selected_text, sentiment = record.text, record.selected_text, record.sentiment

In [469]:
print(tweet)
print(selected_text)
print(len(tweet))

 vocï¿½ que sumiu forever do msn.
vocï¿½ que sumiu forever do msn.
33


In [476]:
out1 = process_tweet(tweet, selected_text, sentiment, tokenizer, max_len)
out2 = process_data(tweet, selected_text, sentiment, tokenizer, max_len)

 vocï¿½ que sumiu forever do ms


In [477]:
out1["token_start"], out1["token_end"]

(4, 14)

In [478]:
out2["targets_start"], out2["targets_end"]

(4, 14)

In [486]:
sum(out2["mask"]), sum(out1["attention_mask"])

(18, 18)

In [473]:
e = bpe_tokenizer.encode(tweet)

In [475]:
for i, (o, token) in enumerate(zip(e.offsets, e.tokens)):
    print(tweet[o[0]: o[1]], "--", token)

 voc -- Ġvoc
ï¿ -- Ã¯
½ -- Â
  -- ¿
qu -- Â½
e su -- Ġque
miu  -- Ġsum
fo -- iu
rever do -- Ġforever
 ms -- Ġdo
n. -- Ġms
 -- n
 -- .


In [488]:
tweet.isascii()

False

In [495]:
(train
 .dropna() 
 .reset_index(drop=True)
 .assign(allascii=lambda x: x.text.apply(lambda x: x.isascii()))
 .allascii
 .sum()
)

27324

In [496]:
(test
 .dropna() 
 .reset_index(drop=True)
 .assign(allascii=lambda x: x.text.apply(lambda x: x.isascii()))
 .allascii
 .sum()
)

3520

In [497]:
train.index.size

27480

In [498]:
test.index.size

3534

In [500]:
(27480 - 27324) / 27480

0.005676855895196507

In [ ]:
156

In [471]:
out1

{'ids': array([    0,  7974,     2,     2, 28312, 29667,  4056,  9470, 14989,
         1192,  6797,  9060,  6000,   109, 43601,   282,     4,     2,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0], dtype=int32),
 'attention_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [460]:
for idx in train.dropna().reset_index(drop=True).index:
    if idx not in [644, 192, 853]:
        print(idx)
        record = train.iloc[idx]
        tweet, selected_text, sentiment = record.text, record.selected_text, record.sentiment
        out = process_tweet(tweet, selected_text, sentiment, tokenizer, max_len)

0
 i`d have responded, if i were going
1
 sooo sad
2
 bullying me
3
 leave me alone
4
 sons of ****,
5
 http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth
6
 fun
7
 soooo high
8
 both of you
9
 wow... u just became cooler.
10
 as much as i love to be hopeful, i reckon the chances are minimal =p i`m never gonna get my cake and stuff
11
 like
12
 dangerously
13
 lost
14
 test test from the lg env2
15
 uh oh, i am sunburned
16
 *sigh*
17
 sick
18
 gonna
19
 hes just not that into you
20
 oh marly, i`m so sorry!! i hope you find her soon!! <3 <3
21
 interesting.
22
 is cleaning the house for her family who is comming later today..
23
 gotta restart my computer .. i thought win7 was supposed to put an end to the constant rebootiness
24
 see wat i mean bout foll0w friidays... it`s called lose f0llowers friday... smh
25
 the free fillin` app on my ipod is fun, im addicted
26
 i`m sorry.
27
...no internet
28
 fun
29
 power back up not working too
30
 q

325
 it was amazin
326
 now a migraine.
327
 there!
328
 ?sucks!?..
329
 whens the sway sway winner announced?
330
 k will check it out...
331
 doing pretty well,
332
 i want to wake up early, and get a coffee tomorrow (today) ! it`s going to be a busyy day! but have to keep writing.. booo whoo!
333
 i miss daddy and mommy
334
 officially depressed
335
 i just realized that i can`t forward text msgs with my iphone.
336
 exciting!
337
 that sucks!
338
 bored
339
 i am lost.
340
 it`s a peter & gordon morning -> and i, go to pieces and i wanna hide / go to pieces and i almost die / ever... ? http://blip.fm/~5yk38
341
 bad
342
 lmao... smh! that one threw me off
343
 oh nice going!
344
 is getting ready for work... working all weekend
345
 partying
346
 i agree with you!
347
 am hopeless at everything else
348
perfect match
349
 thanks!
350
 sorry
351
 i saw amazing heeels. but they were too big
352
 stuck
353
 dang
354
 i missed all ur tweets
355
 damnit all. that sucks.
356
 my boss. sh

 so sorry
541
 i feel for you i hope it`s a smooth flight. be safe
542
 can`t sleep...
543
 sad.
544
 sweet dreams
545
 hay wats ur aim? we should chat
546
 . okay kool... i might be touring all summer long but we can make it happen!
547
 pain.
548
 time to watch op, then some dead like me, then sleep.
549
 having dinner with my grandma since i couldn`t be with my mum.
550
 haha thanks its for history and its on how the invention of television has influence america lol
551
 yea so wassup
552
 stupid
553
 shopped til i dropped....come bac sunshine i miss u
554
 like
555
 thank you i`ve had anxiety issues
556
 around 3 hours left until a 3 day weekend and i have waaaay too much work to do.
557
 loved!
558
 there were attempts to somehow extend inner classes, which would be close to #closure, can`t find the ref atm
559
 yes! join us! all we require is a cupcake donation...
560
 hard
561
 terrible headache and super swollen and puffy eyes! i dont think im going out today.ugh!
562
 can`t ac

 much fun
809
 mitchel you have no idea how much i want to call you!! but it costs loads to call from england!!!
810
 stupid me,
811
 very relaxing
812
 yessir that is 100% right
813
 night!
814
 great
815
 sad.
816
 too warm!!!
817
 ada acara menarik lain: keys to the vip di channel v. ttg straight yg dikasih tasks to approach strangers in the crowd.. hmm
818
 well hit me and we can see...it depends then
819
 thanx
820
 very cute
821
 good
822
 shower time!!!!!
823
 coooooooool... dooooooooown... patience... is... virtue...
824
 gotta drop off some car parts for a buddy to press out, then to the dentist! http://yfrog.com/0a7v3j
825
 i cant call im at work
826
 expensive.
827
 so tired &
828
 happy
829
 am on here..finally. and yay. seriously. that made my day oh fellow hell ruler
830
 goodnight
831
 doesnt have a hangover and is getting ready for a good ol` english fry up
832
 waiting to put my story about stereo skyline up. don`t know where to put it. boooo
833
 working now
834
 just

 memo it looks amazing
1077
 yeah twitter is a lot of fun babe... especially when you`ve got ppl to talk to! lol & that sounds goooooood lol
1078
 loveeee
1079
 so grateful!!
1080
 call me when you have time and a good phone i wanna catch up with you! i love you and miss you.
1081
 has heart burn
1082
 thanks to you that
1083
 even if i was drunk, i`d think about using a fake/temporary mail address
1084
 happy
1085
 luv
1086
 just touched down in changi. +1/2 hrs to kill before heading to penang. must fly singapore air more often!
1087
 afraid
1088
 wow i have no friends on twitter
1089
 hard
1090
 positivity is doin me good! change it up! won`t regret it... forsure..
1091
 my friend sent u call bck messages. i dunno if you could reply but she has no credit thts all.
1092
 love
1093
 laughing
1094
 when leaving the shops i had a medium light coffee frappuccino. nom nom nom
1095
....i`ve become poor again
1096
 uploading pictures on myspace
1097
 but it was kinda funny, sorry. ur still 

...

 oh ok cool will get back to you on that
26362
 i quite like god save our king too
26363
i hope
26364
 yall partyin with out me
26365
 the broiler at burger king is down. there will be no burgers at burger king at lunch! at least chicken fries were there!
26366
 cool
26367
 huh. i`ll have to try that (for no other reason then just to try it
26368
 creep.
26369
 bye.
26370
 ****
26371
 did 15 mins on expresso #cycling (3 miles) and beat my last time by 40 seconds. then lower body weights. now ready for work!
26372
 that would be soooooo much and geeky to the ultimate level!
26373
 best
26374
 good
26375
 it was fun off too
26376
 isn`t so bad when
26377
 voyed on mtv movie awards mostly on twilight and slumdog http://is.gd/wbt6
26378
 i started having anxiety
26379
 i should be paying my bills and not playing on here.
26380
 awesome
26381
 hard!
26382
 there`s nothing wrong with having a quiet day... it gives you more time to listen
26383
 we can`t wait to see you!
26384
 history projec

 _370 i thought he was off until next week...i think we have another week to do his ohmygod please tell me with have another week
26557
 oh snap!
26558
 cavs do it again!! oh yea!
26559
 i love you more than you`ll ever know.
26560
i hate wen i cant sleep.
26561
 sorry
26562
 it all not a nice feeling
26563
 tired
26564
 horrible
26565
 the renegades last day was today and...well... he`s gone... didn`t catch him gutted... closest he came to greenock was kilmacolm!
26566
 well, i was under the impression most people have the urge to buy new sofas to be fair, we`re not tiling, we have a man for that
26567
 happy mothers day to all mothers
26568
 drinking coffee....mmmmm.....coffee
26569
 starting to get sad.
26570
 dang having to work saturday mornings
26571
 then i guess that makes me dot?
26572
 special people.
26573
 soooo far away.
26574
 not sick,
26575
 i just had a horrible dream.
26576
 its too sunny for work !!!
26577
 time to get off the puter. nite all.
26578
 dead
26579
 hate

 i did
26939
 i`m up way to early today! work later.
26940
 my husband thinks my twittering makes me a creeper...
26941
 thanks
26942
 dammit
26943
...i`m really gna miss it
26944
 has been told shes not allowed in the sun tomorow! either that or wear sun cream!
26945
 too hard to please all of my friends at once
26946
 beautiful
26947
 have fun see
26948
 glad
26949
 hahahaha
26950
 poor little core duo just can`t keep up
26951
 missing my man,
26952
 epic haha
26953
 92.7 is turning it right about now.
26954
 people are not twitting very much today
26955
 yes - it has put a damper on us 'getting to know' georgetown! instead he is watching tv and i am catching up on invoices.
26956
 -- no, i have not! glad to hear it was incredible, though! i still have it bookmarked to try sometime
26957
 cute
26958
 me happy & motivated
26959
 happy birthday.
26960
 it`s 12:30am, stopped tweeting, my brain`s not functioning. i wanna cry. haha. philippines tour, please?
26961
 princess_i have to pack

 who`s going out tonight? i want to go out ****
27162
 tired
27163
 i`m about to munch on some more high calorie things! they keep my brain working
27164
 bad
27165
ive been wanting 2 but will be gone 4 2 mo. & have to close my shops!
27166
 awesoome!!!!
27167
 http://twitpic.com/2xjoc - can you paint me
27168
 i want a dog...
27169
 if you hit a car .. u should leave a note http://bit.ly/p274b but yea put whatever u want on the note
27170
 i`ve been to rome, italy. i love it there. elegance, style. very difficult to live in chicago. no elegance here. sad.
27171
 loves
27172
 cant sleep
27173
 blimey!
27174
 can`t you write a song without your guitar? i thought you could look for something to do!
27175
 follow friday.. follow me coz i only have 26 followers... (it`s not 2am)
27176
 great to finally meet you last night! wish we had more time to talk. can`t go to the coffee gtgs, though... day job.
27177
 bad car accident...
27178
 i look forward
27179
 haha!! someone with an axe just po

 getting ready for my first day at my new job
27380
 you lvoe me!
27381
 happy sammy is going 2 here 2morrow!! but unfortunatly i`m not gone be able 2 go 2 the club he performing @
27382
 home after a while out! i feel like a long skate seshion but got arrands to do
27383
 this is a much better tool
27384
 hate
27385
 much appreciate
27386
 thanks.
27387
 sleep. work tomorrow.
27388
 its cloudy outside
27389
 safe
27390
 you stop it
27391
 thanks al...i had a freakin ball!! great company, awesome venues....what more could a girl want? love ya!!
27392
 i figured you`d be bouncing about it
27393
 glad to see and more red bull air racers here keeping us in the loop
27394
 think i have sunstroke.
27395
 shocking...
27396
 favorite!
27397
 haha that`s way cool!
27398
 please remember you need a helmet for the bike asap, talk to you later ok hope your head not to bad
27399
 yay!
27400
aja, enjoyyitverymuch
27401
 wolverine was much fun, but gambit was a bit disappointing. neither smooth nor 

In [422]:
bpe_tokenizertweet

' vocï¿½ que sumiu forever do msn.'

In [332]:
tweet = " Trying to learn how to do this 'twitter' thing... I`ll get there. Welcome to all my followers, though Im not that interesting"
tweet = " " + " ".join(tweet.split())
print(tweet)
e = bpe_tokenizer.encode(tweet)

 Trying to learn how to do this 'twitter' thing... I`ll get there. Welcome to all my followers, though Im not that interesting


In [333]:
total = 0
for token, offsets in zip(e.tokens, e.offsets):
    total += len(token)
    print(token, len(token), total, offsets)

Ġtrying 7 7 (0, 7)
Ġto 3 10 (7, 10)
Ġlearn 6 16 (10, 16)
Ġhow 4 20 (16, 20)
Ġto 3 23 (20, 23)
Ġdo 3 26 (23, 26)
Ġthis 5 31 (26, 31)
Ġ' 2 33 (31, 33)
twitter 7 40 (33, 40)
' 1 41 (40, 41)
Ġthing 6 47 (41, 47)
... 3 50 (47, 50)
Ġi 2 52 (50, 52)
` 1 53 (52, 53)
ll 2 55 (53, 55)
Ġget 4 59 (55, 59)
Ġthere 6 65 (59, 65)
. 1 66 (65, 66)
Ġwelcome 8 74 (66, 74)
Ġto 3 77 (74, 77)
Ġall 4 81 (77, 81)
Ġmy 3 84 (81, 84)
Ġfollowers 10 94 (84, 94)
, 1 95 (94, 95)
Ġthough 7 102 (95, 102)
Ġim 3 105 (102, 105)
Ġnot 4 109 (105, 109)
Ġthat 5 114 (109, 114)
Ġinteresting 12 126 (114, 126)


In [291]:
max_len = 96

ids = np.zeros((max_len), dtype=np.int32)
attention_mask = np.zeros((max_len), dtype=np.int32)
token_type_ids = np.zeros((max_len), dtype=np.int32)

idx = 27478
record = train.iloc[idx]



tweet = " " + " ".join(record.text.split())
sentiment = record.sentiment
selected_text = record.selected_text

# tweet = "this is a good day definitely"
# selected_text = "od day"

encoded = bpe_tokenizer.encode(tweet)



print(tweet)

 Yay good for both of you. Enjoy the break - you probably need it after such hectic weekend Take care hun xxxx


Yay good for both of you.


 Y 1 0 2 0
. 26 25 26 7
0 7
 yay good for both of you.


In [247]:
selected_text.lower()

'good day'

In [217]:
bpe_tokenizer.decode(encoded.ids[token_start:token_end])

' yay good for both of you'

In [151]:
bpe_tokenizer.decode(encoded.ids[token_start:token_end])

' good day'

In [155]:
selected_text.strip() in bpe_tokenizer.decode(encoded.ids[token_start:token_end]).strip()

True

In [99]:
tweet = "this is a good day"
selected_text = "od day"
out = process_data(tweet, selected_text, sentiment, bpe_tokenizer, max_len)

In [100]:
print(out["orig_tweet"])
print(out["orig_selected"])

 this is a good day
 od day


In [88]:
out.get("targets_start"), out.get("targets_end")

(7, 8)

In [90]:
out.keys()

dict_keys(['ids', 'mask', 'token_type_ids', 'targets_start', 'targets_end', 'orig_tweet', 'orig_selected', 'sentiment', 'offsets'])

In [95]:
for idd in out["ids"]:
    if idd != 0:
        print(bpe_tokenizer.decode([idd]))

 positive
</s>
</s>
 this
 is
 a
 good
 day
</s>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>


In [70]:
for i, (offset_start, offset_end) in enumerate(encoded.offsets):
    print("x" + tweet[offset_start: offset_end])

x Y
xay
x good
x for
x both
x of
x you
x.
x Enjoy
x the
x break
x -
x you
x probably
x need
x it
x after
x such
x he
xctic
x weekend
x 
x Take
x care
x hun
x x
xxxx


In [53]:
for i, char in enumerate(tweet):
    if char == selected_text[0]:
        if tweet[i:i+len(selected_text)] == selected_text:
            char_start = i
            char_end = char_start + len(selected_text)

In [54]:
char_start, char_end

(1, 26)

In [57]:
" ".join(tweet.split())[0:2]

'Ya'

In [64]:
t = "this is a      string with many spaces"
e = bpe_tokenizer.encode(t)

In [338]:
len_selected_text = len(" " + selected_text)
print(selected_text)

for offset_start, offset_end in encoded.offsets:
    print(tweet[offset_start:offset_start + len_selected_text])
    if " " + selected_text == tweet[offset_start:offset_start + len_selected_text]:
        start_index = offset_start
        end_index = start_index + len_selected_text
        break

Yay good for both of you.
 Yay good for both of you.


In [340]:
for 

(0, 26)

In [343]:
tweet[start_index: end_index][1:]

'Yay good for both of you.'

In [344]:
selected_text

'Yay good for both of you.'

In [307]:
bpe_tokenizer.decode([0, 1, 2])

'<s><pad></s>'

In [65]:
tokenizers.ByteLevelBPETokenizer?

In [502]:
from sklearn.model_selection import KFold

In [520]:
kfold = KFold(n_splits=5, shuffle=True, random_state=2)

In [521]:
kfold.get_n_splits(X=np.arange(train.shape[0]))

5

In [523]:
for split in kfold.split(np.arange(train.index.size)):
    left, right = split
    print(left.shape, right.shape)

(21984,) (5496,)
(21984,) (5496,)
(21984,) (5496,)
(21984,) (5496,)
(21984,) (5496,)


In [524]:
train.iloc[right]

,textID,text,selected_text,sentiment
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive
12,74a76f6e0a,My Sharpie is running DANGERously low on ink,DANGERously,negative
24,0c8cc71c46,SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cA...,SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cA...,neutral
25,e00c6ef376,"the free fillin` app on my ipod is fun, im add...","the free fillin` app on my ipod is fun, im add...",positive
36,2724775d6b,Born and raised in NYC and living in Texas for...,miss,negative
...,...,...,...,...
27446,a9f0aa00e2,is having carbonara for dinner made by my mum mom,is having carbonara for dinner made by my mum mom,neutral
27448,46b2ecf52e,haha malamang as one of my chef.,haha malamang as one of my chef.,neutral
27455,d32efe060f,i wanna leave work already! Not feelin it 2day,wanna leave work al,negative
27464,c14a543497,Sure. I`ll try n keep that up! =P You enjoy s...,enjoy,positive
